# Machine Learning pour prédire la qualité environnementale d'un aliment 

Dans ce notebook, vous trouvez notre code d'entraînement et validation automatique d'algorithmes de Machine Learning pour prédire la qualité environnementale d'un aliment à partir du jeu de données *AGRIBALYSE® - Synthèse*.

**dataset:** https://www.data.gouv.fr/fr/datasets/agribalyse-r-synthese-1/<br>
**tâche:** Régression<br>
**target variable:** *DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)*

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
#pour l'entraînement automatique:
from supervised.automl import AutoML
#pour générer un rapport html des résultats:
import IPython
import markdown

## 1. Importation des données

On importe le dataset à partir de son url.

In [3]:
url = 'https://www.data.gouv.fr/fr/datasets/r/c763b24a-a0fe-4e77-9586-3d5453c631cd'
data_agrib = pd.read_csv(url)

## 2. Train/test splitting

On n'effectue pas de séléction de variables. On définit la variable réponse et les variables explicatives.

In [4]:
y = data_agrib["DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)"].values
X = data_agrib.drop(columns=["DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)"])

On partage le dataset en dataset d'entraînement et de test. Attention, AutoML nécessite d'un partage 75%/25%.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

## 3. AutoML

On entraîne, test et valide automatiquement plusieurs algorithmes de ML sur le dataset avec AutoML (plus d'infos ici: https://supervised.mljar.com/) et on génère un html des métriques et des paramètres du modèle.

In [ ]:
automl = AutoML(total_time_limit=5*60,mode='Explain',random_state=42,ml_task='regression')
#fit model
automl.fit(X_train, y_train)
#predictions
predictions = automl.predict(X_test)
#generate html report
automl.report()

### Preprocessing fait par AutoML

Il n'est pas évident de comprendre, à partir du rapport html qui est généré, quelles tâches de preprocessing sont faites dans la **pipeline de AutoML** qui permet d' aboutir aux résultats observés.  <br>
Pour mieux comprendre les résultats, on peut s'intéresser au fichier `framework.json` de chaque modèle. Par exemple, on peut consulter ce fichier pour le modèle `Xgboost` ici:
https://github.com/etalab-ia/DGML/blob/main/docs/automodels/c763b24a-a0fe-4e77-9586-3d5453c631cd/4_Default_Xgboost/framework.json<br>
On observe:
1. L'**encoding** des variables catégorielles est fait en transformant le nom de chaque catégorie en un entier
2. Les colonnes contenant du long **texte** sont transformées à l'aide de [`TfidVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

Ces informations sont disponibles pour tous les algorithmes.